In [1]:
from datasets import ARCDataset, ColorPermutation
from itertools import permutations
import numpy as np
import pandas as pd
from copy import copy

In [2]:
perm = ColorPermutation()
ds = ARCDataset()
ds = ARCDataset(tasks=ds.tasks[:10], augs=[perm])

In [3]:
len(ds)

8880

In [7]:
%%timeit

ds[8879]

708 µs ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
